# M5 Forecasting - Accuracy

Соревнование на платформе Kaggle для предсказания продаж по различным продуктам сети Walmart на 28 дней вперед.

Набор данных включает в себя данный о продажах 3049 продуктов, классифицированных на 3 категории: Хобби, Продукты питания и Товары для дома, и разделенных еще на 7 категорий.
Продукты продаются в 10 магазинах 3 штатов (Калифорния, Техас и Вайоминг). 

## Файлы

* <code>calendar.csv</code> Содержит информацию о датах продажи товаров.

* <code>sales_train_validation.csv</code> Содержит ежедневные данные о продажах товаров в каждом магазине 

* <code>sample_submission.csv</code> Пример предоставления данных.

* <code>sell_prices.csv</code> Содержит информацию о ценах по дням.

Файл 'calerndar.csv' включает информацию о датах продаж:
* <code>date</code> - Дата в формате 'y-m-d'
* <code>wm_yr_wk</code> - Идентификатор недели, к которому относится дата
* <code>weekday</code> - День недели (понедельник, вторник, ...)
* <code>wday</code> - Идентификатор дня недели,начиная с воскресенья
* <code>month</code> - Месяц
* <code>year</code> - Год
* <code>d</code> - Номер дня
* <code>event_name_1</code> - Название праздника
* <code>event_type_1</code> - Тип праздника
* <code>event_name_2</code> - Название праздника,  если он не вошел в event_name_1
* <code>event_type_2</code> - Тип праздника, если он не вошел в event_name_1
* <code>snap_CA, snap_TX, snap_WI</code> - Бинарная переменная (0, 1), указывющая, осуществляются ли в указанную дату в магазинах Калифорнии, Техаса и Вайоминга покупки SNAP. SNAP - продовольственная программа правительства США по выдаче бесплатных продуктов 

Файл 'sales_train_validation.csv' содержит информацию о ежедневных данных о продажах на единицу товара и магазина:
* <code>id</code> - Идентификатор 
* <code>item_id</code> - Идентификатор продукта
* <code>dept_id</code> - Идентификатор вида продукта
* <code>cat_id</code> - Идентификатор категории продукта
* <code>store_id</code> - Идентификатор магазина
* <code>state_id</code> - Штат, в котором расположен магазин
* <code>d_1, d_2, ... , d_1914</code> - Количество покупок в день, начиная с даты 29.01.2011

Файл 'sell_price.csv' содержит информацию о ежедневных ценах на товар:
* <code>store_id</code> - Идентификатор магазина
* <code>item_id</code> - Идентификатор продукта
* <code>wm_yr_wk</code> - Идентификатор недели
* <code>sell_price</code> - Цена продукта

### Импорт модулей

In [1]:
import pandas as pd
import numpy as np
from  datetime import datetime, timedelta
import dateutil.relativedelta
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc
import pickle
import random 
import time
import math

In [2]:
# функция для преобразования типов данных в столбцах
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: 
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Объем памяти снизился на {:5.2f} Mb ({:.1f}%)'\
                      .format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
pd.options.display.max_columns = 50 #увеличесние максимального количества выводимых столбцов

### Загрузка данных 

In [4]:
calendar = pd.read_csv('calendar.csv')
calendar = reduce_mem_usage(calendar)
print('Calendar содержит {} строк и {} колонок'.format(calendar.shape[0], calendar.shape[1]))

sell_prices = pd.read_csv('sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)
print('Sell_prices содержит {} строк и {} колонок'.format(sell_prices.shape[0], sell_prices.shape[1]))

sales = pd.read_csv('sales_train_validation.csv')
print('Sales_train_validation содержит {} строк и {} колонок'.format(sales.shape[0], sales.shape[1]))

Объем памяти снизился на  0.12 Mb (41.9%)
Calendar содержит 1969 строк и 14 колонок
Объем памяти снизился на 130.48 Mb (37.5%)
Sell_prices содержит 6841121 строк и 4 колонок
Sales_train_validation содержит 30490 строк и 1919 колонок


In [5]:
id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
product = sales[id_columns].drop_duplicates()

In [6]:
submission = pd.read_csv('sample_submission.csv')
validate_submission = submission[submission.id.str.endswith('validation')]
eval_submission = submission[submission.id.str.endswith('evaluation')]

newcolumns = ['id'] + ['d_{}'.format(i) for i in range(1914, 1914+28)]
validate_submission.columns = newcolumns
validate_submission = validate_submission.merge(product, how = 'left', on = 'id')

In [7]:
id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# использую данные только за последние 2 года
DAYS = 365*2; LAST_DAY=1913
day_columns = ["d_{}".format(i) for i in range(LAST_DAY-DAYS+1, LAST_DAY+1)]
print(len(day_columns), day_columns[0])
sales = sales[id_columns + day_columns]
print(sales.shape)
sales.head()

730 d_1184
(30490, 736)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1184,d_1185,d_1186,d_1187,d_1188,d_1189,d_1190,d_1191,d_1192,d_1193,d_1194,d_1195,d_1196,d_1197,d_1198,d_1199,d_1200,d_1201,d_1202,...,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,1,0,0,0,0,0,2,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,4,1,1,1,1,1,2,14,2,2,0,2,1,2,3,3,2,2,...,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,1,2,0,0,0,0,3,1,0,0,0,1,0,2,1,1,2,2,...,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [8]:
# функция преобразования датафрейма
# подсчитывает количество продаж по каждому виду товара за определенный день
def melted(df, name=''):
    df = pd.melt(df, id_vars = id_columns, var_name = 'day', value_name = 'demand')
    print('{}: {} строк и {} колонок'.format(name, df.shape[0], df.shape[1]))
    df = reduce_mem_usage(df)
    return df

melted_sales = melted(sales)
melted_sales['part'] = 'train'
melted_validate = melted(validate_submission)
melted_validate['part'] = 'validate'

: 22257700 строк и 8 колонок
Объем памяти снизился на 1231.14 Mb (9.4%)
: 853720 строк и 8 колонок
Объем памяти снизился на 46.41 Mb (10.9%)


In [9]:
data = pd.concat([melted_sales, melted_validate], axis = 0)
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1184,0,train
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1184,0,train
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1184,0,train
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1184,1,train
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1184,0,train


In [10]:
del melted_sales, melted_validate
del submission, validate_submission, eval_submission, product
del sales,
gc.collect()

56

In [11]:
# объединение всех загруженных и преобразованных датафреймов в финальный датафрейм
calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d', 'day'], inplace = True, axis = 1)

data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'])

print('Трейн содержит {} строк и {} столбцов'.format(data.shape[0], data.shape[1]))

Трейн содержит 22589864 строк и 18 столбцов


In [12]:
del calendar, sell_prices
gc.collect()

40

### Создание новых признаков и преобразование категориальных переменных

In [13]:
# функция преобразовывает категориальные переменные в целые числа (One-hot-encoding)
def encoding(df, list):
    for column in list:
        df[column] = df[column].astype("category").cat.codes.astype("int16")
        df[column] -= df[column].min()    
    return df

event_dtypes = ['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2']

id_dtypes = ['store_id', 'item_id', 'state_id', 'dept_id', 'cat_id']

event_fillna = {'event_name_1': 'normal', 
                'event_name_2': 'normal',
                'event_type_1': 'normal', 
                'event_type_2': 'normal'}

data = encoding(data, id_dtypes).pipe(reduce_mem_usage)
data.fillna(value=event_fillna, inplace = True)
data = encoding(data, event_dtypes).pipe(reduce_mem_usage)

Объем памяти снизился на 1701.93 Mb (4.8%)
Объем памяти снизился на 1098.71 Mb (7.3%)


In [14]:
# создание новых признаков из даты 
# 'year' - год
# 'quarter' - квартал
# 'month' - месяц
# 'week' - неделя
# 'day' - день
# 'dayofweek' - день недели
# 'weekday' - номер дня недели
# 'weekofyear' - номер недели в году
# 'is_weekend' - выходной
def datetime_features(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    attrs = ['year','quarter', 'month', 'week', 
             'day', 'dayofweek', 'weekday','weekofyear']
    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        df[attr] = getattr(df['date'].dt, attr).astype(dtype)
    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8)
    return df

data = datetime_features(data)
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,year,quarter,month,week,day,dayofweek,weekday,weekofyear,is_weekend
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2014-04-26,11413,30,4,1,1,0,0,0,8.257812,2014,2,4,17,26,5,5,17,1
1,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,train,2014-04-27,11413,30,4,1,1,0,0,0,8.257812,2014,2,4,17,27,6,6,17,1
2,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2014-04-28,11413,30,4,1,1,0,0,0,8.257812,2014,2,4,18,28,0,0,18,0
3,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,train,2014-04-29,11413,30,4,1,1,0,0,0,8.257812,2014,2,4,18,29,1,1,18,0
4,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2014-04-30,11413,30,4,1,1,0,0,0,8.257812,2014,2,4,18,30,2,2,18,0


In [15]:
data.sort_values(by=['id', 'date'], inplace=True)

X_train = data[data['part'] == 'train']
X_val = data[data['part'] == 'validate']

print(len(X_train), len(X_val))
del data; gc.collect()

21736144 853720


135

In [16]:
# сохранение объектов X_train и X_val
dbfile = open('X_train.pkl', 'wb') 
pickle.dump(X_train, dbfile)
dbfile.close()

dbfile = open('X_val.pkl', 'wb') 
pickle.dump(X_val, dbfile)
dbfile.close() 

In [17]:
# функция для создания скользящих окон в 7 и 28 дней (неделя и месяц)
def numerical_feature(df):
    for i in [7, 28]:
        df[f"shifted_t{i}"] = df[["id","demand"]].groupby('id')["demand"].shift(i)

    for win, col in [(7, "shifted_t7"), (7, "shifted_t28"), (28, "shifted_t7"), (28, "shifted_t28")]:
        df[f"rolling_mean_{col}_w{win}"] = df[["id", col]].groupby('id')[col].shift(1).rolling(win, min_periods=1).mean()
    return df



In [18]:
X_train = numerical_feature(X_train)
X_train.dropna(inplace = True)
gc.collect()

Wall time: 27 s


0

In [19]:
X_train.shape, X_train.columns

((20851934, 33),
 Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'demand',
        'part', 'date', 'wm_yr_wk', 'event_name_1', 'event_type_1',
        'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
        'sell_price', 'year', 'quarter', 'month', 'week', 'day', 'dayofweek',
        'weekday', 'weekofyear', 'is_weekend', 'shifted_t7', 'shifted_t28',
        'rolling_mean_shifted_t7_w7', 'rolling_mean_shifted_t28_w7',
        'rolling_mean_shifted_t7_w28', 'rolling_mean_shifted_t28_w28'],
       dtype='object'))

### Разделение датафрейма на обучающий набор и набор для проверки

In [20]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id',
             'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2']
useless_cols = ['id', 'part', 'date', 'demand', 'd', 'wm_yr_wk', 'weekday']
train_cols = X_train.columns[~X_train.columns.isin(useless_cols)]

y_train = X_train['demand']
X_train = X_train[train_cols]

In [21]:
%%time
from sklearn.model_selection import train_test_split
np.random.seed(777)

X, x_test, Y, y_test = train_test_split(X_train, y_train, test_size=0.05)

Wall time: 7.12 s


In [22]:
del X_train, y_train; gc.collect()

90

### LightGBM модель

In [23]:
parameters = {'objective': 'poisson',
              'force_row_wise': True,
              'learning_rate': 0.075,
              'sub_row': 0.75,
              'bagging_freq': 1,
              'lambda_l2': 0.1,
              'metric': ['rmse'],
              'verbosity': 1,
              'num_iterations' : 1000,
              'num_leaves': 128,
              'min_data_in_leaf': 100}

In [24]:
train_set = lgb.Dataset(X, Y)
test_set = lgb.Dataset(x_test, y_test)
del X, Y, x_test, y_test; gc.collect()

40

In [25]:
dbfile = open('train_set.pkl', 'wb') 
pickle.dump(train_set, dbfile)
dbfile.close() #Dont forget this 

dbfile = open('test_set.pkl', 'wb') 
pickle.dump(test_set, dbfile)
dbfile.close() #Dont forget this 

In [26]:
%%time

model = lgb.train(parameters, train_set, valid_sets = [test_set], verbose_eval = 50)

C:\Users\Miron\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[50]	valid_0's rmse: 2.26255
[100]	valid_0's rmse: 2.20777
[150]	valid_0's rmse: 2.18987
[200]	valid_0's rmse: 2.17645
[250]	valid_0's rmse: 2.16529
[300]	valid_0's rmse: 2.1576
[350]	valid_0's rmse: 2.14901
[400]	valid_0's rmse: 2.14165
[450]	valid_0's rmse: 2.13607
[500]	valid_0's rmse: 2.1304
[550]	valid_0's rmse: 2.12655
[600]	valid_0's rmse: 2.12146
[650]	valid_0's rmse: 2.11857
[700]	valid_0's rmse: 2.11591
[750]	valid_0's rmse: 2.11321
[800]	valid_0's rmse: 2.11045
[850]	valid_0's rmse: 2.10791
[900]	valid_0's rmse: 2.10601
[950]	valid_0's rmse: 2.1037
[1000]	valid_0's rmse: 2.10097
Wall time: 18min 5s


In [27]:
model.save_model("model.lgb")

### Предсказание

In [28]:
with open("X_train.pkl", 'rb') as fin:
    X_train = pickle.load(fin)

with open("X_val.pkl", 'rb') as fin:
    X_val = pickle.load(fin)

In [29]:
%%time
last_date = '2016-01-29' # - 86 дней от последней даты датафрейма
X_train = X_train[X_train['date'] >= last_date]

Wall time: 476 ms


In [30]:
# функция для создания тестового датафрейма и получения предсказания с помощьюю обученной модели
# factor - веса
def predict(model, X_train, X_test, factor=1):
    dates = X_test['date'].unique()
    count_dates = len(dates)
    print("Прогноз для ", count_dates, 'дат')
    
    col = ['id'] + ['F{}'.format(i) for i in range(1, count_dates+1)] #создание столбцов вида F1 - F28
    dept_id = X_train['dept_id'].unique() #уникальные виды продуктов (dept_id)
    
    accuracies = []
    dept_id = sorted(dept_id)
    for iid in dept_id:
        test = X_test[X_test['dept_id']==iid]
        ids = test['id'].unique() #уникальные значения по видам продукта (item_id)
        ids_zeros = np.zeros((len(ids), count_dates+1))
        ids_df = pd.DataFrame(ids_zeros, columns=col)
        ids_df['id'] = test[test['date']==dates[0]]['id'].values
        train = X_train[X_train['dept_id']==iid]
        lastmonth = pd.to_datetime(train.head(1)['date'])
        
        for idx, date in enumerate(dates):
            newrow = test[test['date']==date]
            train = train.append(newrow)
            train.sort_values(by=['id', 'date'], inplace=True)
            feat = numerical_feature(train)
            x = feat.loc[feat['date']==date , train_cols]
            val_pred = model.predict(x, num_iteration=model.best_iteration)
            ids_df[f'F{idx+1}'] = val_pred * factor
            train.loc[train['date']==date, 'demand'] = val_pred * factor
            lastmonth = lastmonth + pd.DateOffset(days=1)
            train = train[train['date'] >= str(lastmonth.values[0])]
            
        accuracies.append(ids_df)
        accuracies = [pd.concat(accuracies)]
        print(iid)

    accuracies = pd.concat(accuracies)
    return accuracies

In [31]:
final_pred = []
weights = [1, 1.028, 1.023, 1.018] #веса
for w in weights:
    print('Получение предсказаний с весом -',w)
    pred = predict(model, X_train, X_val, factor=w)
    final_pred.append(pred)

Получение предсказаний с весом - 1
Прогноз для  28 дат


C:\Users\Miron\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


0
1
2
3
4
5
6
Получение предсказаний с весом - 1.028
Прогноз для  28 дат
0
1
2
3
4
5
6
Получение предсказаний с весом - 1.023
Прогноз для  28 дат
0
1
2
3
4
5
6
Получение предсказаний с весом - 1.018
Прогноз для  28 дат
0
1
2
3
4
5
6


In [32]:
#усреднение полученных предсказаний
avg_pred = pd.DataFrame([])
avg_pred['id'] = final_pred[0]['id']
for i in range(1, 29):
    avg_pred[f'F{i}'] = (final_pred[1][f'F{i}'] + final_pred[2][f'F{i}'] + final_pred[3][f'F{i}'])/3

In [33]:
submission = pd.read_csv(r'sample_submission.csv')
dfeval = submission[submission.id.str.endswith('evaluation')]
assert len(dfeval)==len(avg_pred)

In [36]:
#функция для сохранения полученных данных
def save_csv(d):
    df = pd.concat([d, dfeval]) 
    df.reset_index(drop=True, inplace = True)
    return df.to_csv(f'submission.csv', index=False)

save_csv(avg_pred) 